In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
spam = pd.read_csv("../input/sms-spam-collection-dataset/spam.csv", encoding='ISO-8859-1')

print(spam.shape)
spam.head()

(5572, 5)


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
# trim table to have only 2 columns: label (ham/spam), sms text
spam=spam.iloc[:,:2]
print(spam.shape)
spam.columns=['label','sms']
spam.head()

(5572, 2)


,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
spam.describe()

,label,sms
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [6]:
spam['contains_free'] = 0

spam_free = 0
ham_free = 0

for index, row in spam.iterrows():
    if 'free' in row['sms'].lower():
        spam.at[index, 'contains_free'] = 1
        
        if row['label'] == 'spam':
            spam_free += 1
        else:
            ham_free += 1

print(spam)
print("Number of spam messages containing 'free':", spam_free)
print("Number of ham messages containing 'free':", ham_free)

# a lot more spam messages contain the word "free" -- as expected !

     label                                                sms  contains_free
0      ham  Go until jurong point, crazy.. Available only ...              0
1      ham                      Ok lar... Joking wif u oni...              0
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...              1
3      ham  U dun say so early hor... U c already then say...              0
4      ham  Nah I don't think he goes to usf, he lives aro...              0
...    ...                                                ...            ...
5567  spam  This is the 2nd time we have tried 2 contact u...              0
5568   ham              Will Ì_ b going to esplanade fr home?              0
5569   ham  Pity, * was in mood for that. So...any other s...              0
5570   ham  The guy did some bitching but I acted like i'd...              1
5571   ham                         Rofl. Its true to its name              0

[5572 rows x 3 columns]
Number of spam messages containing 'free': 199
Numb

In [7]:
# import nlp library and feed blog, comment, dataset
%pip install --upgrade tensorflow-io

import spacy
nlp = spacy.load("en_core_web_sm")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.31.0
    Uninstalling tensorflow-io-gcs-filesystem-0.31.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0
  Attempting uninstall: tensorflow-io
    Found existing installation: tensorflow-io 0.31.0
    Uninstalling tensorflow-io-0.31.0:
      Successfully uninstalled tensorflow-io-0.31.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
# define a function that converts messages to remove stop words, punctuations: for cleansing
def msg_process(msg_text,toStem=False):
    text = nlp(msg_text)
    result = []
    
    
    # iterate over each token as processed by nlp over the msg_text
    for tkn in text:
        
        # check if the token is a punctuation and if it is a stop word (eg: a,an,the,..)
        if not tkn.pos_ == 'PUNCT' and not tkn.is_stop:
            
            # if we want the msg to be stemmed, we do so
            if toStem == True:
                tkn = tkn.lemma_.lower()
            else:
                # spacy already has an attribute called lower_ !
                tkn = tkn.lower_
            
            # we add the processed message to the result list
            result.append(tkn)
            
    
    # return the contents of the result list separated by a space
    return ' '.join(result)

In [9]:
msg_process('HELLO i am under da water here too much raining ',toStem=True)

'hello da water raining'

In [10]:
spam['binaryLabel'] = spam['label'].map({'ham':0, 'spam':1})
spam.head()

,label,sms,contains_free,binaryLabel
0,ham,"Go until jurong point, crazy.. Available only ...",0,0
1,ham,Ok lar... Joking wif u oni...,0,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,1
3,ham,U dun say so early hor... U c already then say...,0,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,0


In [11]:
spam['sms_trim'] = spam['sms'].apply(msg_process)
spam['sms_trim_lemma'] = spam['sms'].apply(lambda x: msg_process(x,toStem = True))

spam.head()

,label,sms,contains_free,binaryLabel,sms_trim,sms_trim_lemma
0,ham,"Go until jurong point, crazy.. Available only ...",0,0,jurong point crazy available bugis n great wor...,jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,0,0,ok lar joking wif u oni,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,1,free entry 2 wkly comp win fa cup final tkts 2...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,0,0,u dun early hor u c,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,0,nah think goes usf lives,nah think go usf live


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# to transform text into matrix of TF-IDF
tfidf = TfidfVectorizer(decode_error='ignore')

countVector = CountVectorizer(decode_error='ignore')

In [13]:
# both of these are sparse matrices where each row represents a document 
# (sms_trim/sms_trim_lemma) and each column represents a unique term in the corpus
# The values in the matrices represent the TF-IDF weights of the terms in the corresponding documents.

x_tfidf = tfidf.fit_transform(spam['sms_trim'])
x_tfidf_lemma = tfidf.fit_transform(spam['sms_trim_lemma'])

In [14]:
binaryLabelArray = spam['binaryLabel'].values
binaryLabelArray

array([0, 0, 1, ..., 0, 0, 0])

In [15]:
from sklearn.model_selection import train_test_split

# training model based on 80-20 split - 80 percent data for training, 20 percent for testing as suggested 
# on stackoverflow: https://stackoverflow.com/questions/13610074/is-there-a-rule-of-thumb-for-how-to-divide-a-dataset-into-training-and-validatio

# random state is usually 42 everywhere but I put my lucky num 3
x_train_termFreq, x_test_termFreq, y_train_termFreq, y_test_termFreq = train_test_split(
    x_tfidf, binaryLabelArray,test_size=0.2,random_state=3)

# training on the lemmatized data
x_train_lemma, x_test_lemma, y_train_lemma, y_test_lemma = train_test_split(
    x_tfidf_lemma, binaryLabelArray,test_size=0.2,random_state=3)

print("hello")

hello


In [16]:
# Using 3 different algorithms and comparing results

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

algoArray = [LogisticRegression(random_state=3),
            MultinomialNB(),
            DecisionTreeClassifier(random_state=3)]

In [17]:
# method to train the data on each of the algos, and get accuracy on the testing set

from sklearn.metrics import accuracy_score

def trainAlgo(algo, x_train, x_test, y_train, y_test):
    
    # train the algo on the training data
    algo.fit(x_train, y_train)
    
    #predict the testing data
    pred = algo.predict(x_test)
    
    # compare the predicted and actual data and get accuracy score of algo
    return accuracy_score(pred, y_test)

In [18]:
algoNames = []
termFreqScores = []

for algo in algoArray:
    algoNames.append(algo.__class__.__name__)
    termFreqScores.append(trainAlgo(algo, x_train_termFreq, x_test_termFreq, y_train_termFreq, y_test_termFreq))
    df = pd.DataFrame(index = algoNames, data = termFreqScores, columns=['Term Freq Score'])

df

,Term Freq Score
LogisticRegression,0.947982
MultinomialNB,0.971300
DecisionTreeClassifier,0.970404


In [19]:
algoNames = []
termFreqLemmaScores = []

for algo in algoArray:
    algoNames.append(algo.__class__.__name__)
    termFreqLemmaScores.append(trainAlgo(algo, x_train_lemma, x_test_lemma, y_train_lemma, y_test_lemma))
    df_termFreq_lemma = pd.DataFrame(index = algoNames, data = termFreqLemmaScores, columns=['Term Freq Lemma Score'])

termFreqLemmaScores
df_termFreq_lemma

# we notice that MNB and DTC have higher scores than Logistic Regression in general

,Term Freq Lemma Score
LogisticRegression,0.950673
MultinomialNB,0.968610
DecisionTreeClassifier,0.971300


In [20]:
# Now I am going to try to train and test data based on count vectorizing, instead of TF-IDF

In [21]:
# both of these are sparse matrices where each row represents a document 
# (sms_trim/sms_trim_lemma) and each column represents a unique term in the corpus
# The values in the matrices represent simply the frequency of these terms respective to the document

x_countVec = countVector.fit_transform(spam['sms_trim'])
x_countVec_lemma = countVector.fit_transform(spam['sms_trim_lemma'])

In [22]:
x_train_countVec, x_test_countVec, y_train_countVec, y_test_countVec = train_test_split(
    x_countVec, binaryLabelArray,test_size=0.2,random_state=3)

# training on the lemmatized data
x_train_countVec_lemma, x_test_countVec_lemma, y_train_countVec_lemma, y_test_countVec_lemma = train_test_split(
    x_countVec_lemma, binaryLabelArray,test_size=0.2,random_state=3)

In [23]:
algoNames

['LogisticRegression', 'MultinomialNB', 'DecisionTreeClassifier']

In [24]:
# testing with count vectorizer and non-lemmatized

algoNames = []
algoNames
countVecScores = []

for algo in algoArray:
    algoNames.append(algo.__class__.__name__)
    countVecScores.append(trainAlgo(algo, x_train_countVec, x_test_countVec, y_train_countVec, y_test_countVec))
    df_countVec = pd.DataFrame(index = algoNames, data = countVecScores, columns=['Count Vector Score'])

df_countVec

,Count Vector Score
LogisticRegression,0.977578
MultinomialNB,0.976682
DecisionTreeClassifier,0.972197


In [25]:
# testing with count vectorizer and lemmatized

algoNames = []
countVecLemmaScores = []

for algo in algoArray:
    algoNames.append(algo.__class__.__name__)
    countVecLemmaScores.append(trainAlgo(algo, x_train_countVec_lemma, x_test_countVec_lemma, 
                                         y_train_countVec_lemma, y_test_countVec_lemma))
    df_countVec_Lemma = pd.DataFrame(index = algoNames, data = countVecLemmaScores, columns=['Count Vector Lemma Score'])

df_countVec_Lemma

,Count Vector Lemma Score
LogisticRegression,0.979372
MultinomialNB,0.982960
DecisionTreeClassifier,0.967713


In [26]:
# we can see the count vectorized data has much higher scores

In [27]:
allScores = pd.concat([df, df_termFreq_lemma, df_countVec, df_countVec_Lemma], axis = 1)
allScores

,Term Freq Score,Term Freq Lemma Score,Count Vector Score,Count Vector Lemma Score
LogisticRegression,0.947982,0.950673,0.977578,0.979372
MultinomialNB,0.971300,0.968610,0.976682,0.982960
DecisionTreeClassifier,0.970404,0.971300,0.972197,0.967713


In [28]:
allScores.max().sort_values(ascending=True)

Term Freq Score             0.971300
Term Freq Lemma Score       0.971300
Count Vector Score          0.977578
Count Vector Lemma Score    0.982960
dtype: float64

In [29]:
# As we can see, the best scores were obtained by Count Vectorizer with lemmatization, trained, fit and tested
# on the Multinomial Naive-Bayes Algorithm

In [30]:
# Trying hyperparameter tuning

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from datetime import datetime



In [32]:
# params = {'alpha':[0.001,0.01,0.1,0.2,1,2,3,4,5,6,7,10,100]}
params = {
    'alpha': [0.001, 0.01, 0.1, 0.2, 1, 2, 3, 4, 5, 6, 7, 10, 100],
    'fit_prior': [True, False],
    'class_prior': [None, [0.2, 0.8], [0.5, 0.5]]  # Modify the class prior values accordingly
}


grid_search = GridSearchCV(MultinomialNB(), params, cv=10, verbose=True)

grid_search.fit(x_train_countVec_lemma, y_train_countVec_lemma)

print(grid_search.best_params_)
grid_search.best_estimator_

Fitting 10 folds for each of 78 candidates, totalling 780 fits
{'alpha': 3, 'class_prior': None, 'fit_prior': True}


MultinomialNB(alpha=3)

In [33]:
mnb = MultinomialNB(alpha=3, class_prior=None, fit_prior=True)

mnb.fit(x_train_countVec_lemma, y_train_countVec_lemma)
y_pred = mnb.predict(x_test_countVec_lemma)

print("Accuracy: ", accuracy_score(y_pred, y_test_countVec_lemma))
print(classification_report(y_pred, y_test_countVec_lemma))

Accuracy:  0.9883408071748879
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       967
           1       0.94      0.98      0.96       148

    accuracy                           0.99      1115
   macro avg       0.97      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [34]:
mnb = MultinomialNB(alpha=3, class_prior=None, fit_prior=True)

mnb.fit(x_train_countVec_lemma, y_train_countVec_lemma)
y_pred_lemma = mnb.predict(x_test_countVec_lemma)

print("Accuracy: ", accuracy_score(y_pred_lemma, y_test_countVec_lemma))
print(classification_report(y_pred_lemma, y_test_countVec_lemma))

Accuracy:  0.9883408071748879
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       967
           1       0.94      0.98      0.96       148

    accuracy                           0.99      1115
   macro avg       0.97      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [35]:
print("Accuracy: ", accuracy_score(y_pred_lemma, y_test_countVec_lemma))


Accuracy:  0.9883408071748879
